In [5]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import glob
import fastcat as fc

In [7]:
ggems_scatter_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*scatter.mhd'
)
ggems_primary_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*[!scatter].mhd'
)

ggems_scatter_files.sort()
ggems_primary_files.sort()

ggems_primary_projections = []
ggems_scatter_projections = []

for ggems_scatter_file, ggems_primary_file in zip(ggems_scatter_files, ggems_primary_files):
    ggems_prmiary, b, c = fc.utils.read_mhd(ggems_primary_file)
    ggems_scatter, b, c = fc.utils.read_mhd(ggems_scatter_file)

    ggems_primary_projections.append(ggems_prmiary.squeeze())
    ggems_scatter_projections.append(ggems_scatter.squeeze())

In [13]:
plt.figure()
plt.imshow(ggems_primary_projections[23])

In [40]:
# Make an array that is the radial distance from the center of the image
def radial_distance(image,pixel_size=0.784):
    x, y = np.meshgrid(np.arange(image.shape[0]), np.arange(image.shape[1]))
    x = x - image.shape[0] / 2
    y = y - image.shape[1] / 2
    x = x * pixel_size
    y = y * pixel_size
    return np.sqrt(x**2 + y**2)

# Make an array measuring the intensity in the projection as a function of radial distance
def r_squared_falloff(image,SAD):
    r = radial_distance(image)
    return (SAD / (r**2 + SAD**2)**0.5)**3

In [42]:
# Compare the last twenty columns of the images in a plot
proj_only = np.array(ggems_primary_projections) - np.array(ggems_scatter_projections)

plt.figure()
plt.plot(ggems_primary_projections[23][:-40,-20:].mean(axis=1))

r_sq = r_squared_falloff(ggems_primary_projections[23], 1510)
plt.plot(r_sq[:-40,-20:].mean(axis=1) * np.mean(ggems_primary_projections[23][:-40,-20:])/np.mean(r_sq[:-40,-20:]))

In [20]:
plt.figure()
plt.imshow(r_sq)